In [ ]:
%pylab inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [ ]:
from qiskit.tools.monitor import backend_overview
from qiskit.tools.monitor import job_monitor
from qiskit import *

In [ ]:
from qiskit import Aer
sim_backend = Aer.get_backend('qasm_simulator')

## 1. 用H gate產生一組二進位亂數

In [ ]:
#把二進位位元串換成小數
def digit_to_decimal(s):
    b_num = list(s.replace('.',''))[::-1]
    value = 0
    for i in range(len(b_num)):
        digit = b_num.pop()
        if i==0:continue
        if digit == '1':
            value = value + pow(2, -i)
    return value

In [ ]:
# n為最多的位元數目 
n=5
#建構量子電路
def qcirc():
    q=QuantumRegister(n,name='q')
    c=ClassicalRegister(n,name='c')
    circ=QuantumCircuit(q,c)
    circ.h(q)
    circ.barrier()
    circ.measure(q,c)
    return circ

In [ ]:
#執行量子電路，進行一次測量，得到一個二進位的數
circ=qcirc()
circ.draw(output='mpl')
job=execute(circ,backend=sim_backend, shots=1)
job_monitor(job)
s=list(job.result().get_counts().keys())[0]

In [ ]:
# 把二進位帶入自定義的函數，其函數進行二進位->十進位轉換，而得到一個小數。
# 不斷重複執行量子電路，每次都會得到不同的值。
print(s)
N=digit_to_decimal(s)
print(N)

## 2. 用蒙地卡羅法估計$\pi$ 的數值：蒙地卡羅法的概念在於不斷產生亂數，並判斷此亂數是否滿足某條件，最終得到滿足與不滿足某條件的比例。在此我們隨機取一組亂數當為（x,y)座標，若該點滿足$x^2+y^2 <1$，則滿足的數目加一。     
#### Code reference: 'https://medium.com/datadriveninvestor/estimating-pi-using-qiskit-ibms-quantum-computing-framework-495857634e63'  accessed Fed. 2021

In [ ]:
def estimate_pi(n_trials):
    total_counts = []
    for i in range(n_trials*2):
        circuit=qcirc()
        job = execute(circuit,backend=sim_backend,shots=1)
        job_monitor(job)
        result = job.result()
        counts = result.get_counts(circuit)
        total_counts.append(counts)
    res = [list(x.keys())[0] for x in total_counts]
    digits = [ '0.{}'.format(x) for x in res]
    decimals = [digit_to_decimal(x) for x in digits]
    xs = decimals[:int(len(res)/2)]
    ys = decimals[int(len(res)/2):]
    n_couples = len(xs)
    m = ((np.asarray(xs)**2 + np.asarray(ys)**2)<1).sum()
    return 4.*m/n_couples

In [ ]:
x=estimate_pi()
print(x)

## 挑戰題：用matplotlib畫出 $\pi$ 值與取點數目的關係。你有何發現？